# Download National Structure Inventory

In [ ]:
import os
import requests
import urllib.request
import json
import io
import zipfile
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("USA Contiguous Albers Equal Area Conic USGS")
arcpy.env.parallelProcessingFactor = "100%"

data_path = "your_path_here"

## Download data

In [ ]:
### Download all structures
# Base URL for the downloads
base_url = "https://nsi.sec.usace.army.mil/downloads/nsi_2022/nsi_2022_"

# List of states from 1 to 56 (max FIPS state code)
states = [f"{i:02d}" for i in range(1, 57)]

# Location to save the files
location = os.path.join(data_path, "state_gpkgs_zip")

# Loop through the states and download the files
for i in states:
    print(i)
    # Construct the URL for the download
    url = f"{base_url}{i}.gpkg.zip"
    # Construct the file path to save the downloaded file
    file_path = os.path.join(location, f"nsi_2022_{i}.gpkg.zip")
    
    try:
        # Download the file
        urllib.request.urlretrieve(url, file_path)
    except Exception as e:
        # If there's an error, print it and continue
        print(f"Failed to download {url}: {e}")

## Unzip data

In [ ]:
### Unzip the structures
unzipped_folder = os.path.join(data_path, "state_gpkgs")  
os.makedirs(unzipped_folder, exist_ok=True)

# Loop through each ZIP file in the folder
for zip_filename in os.listdir(location):
    if zip_filename.endswith(".zip"):
        zip_path = os.path.join(location, zip_filename)
        
        # Create a new folder to extract to (named after the zip file without the extension)
        extract_folder = os.path.join(unzipped_folder, os.path.splitext(os.path.splitext(zip_filename)[0])[0])
        os.makedirs(extract_folder, exist_ok=True)
        
        try:
            # Extract the ZIP file to the new folder
            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(extract_folder)
            print(f"Successfully extracted: {zip_filename}")
        except Exception as e:
            print(f"Failed to extract {zip_filename}: {e}")


## Add data to geodatabase

In [ ]:
fips_to_abbreviation = {
    '01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', 
    '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', 
    '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', 
    '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', 
    '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', 
    '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', 
    '54': 'WV', '55': 'WI', '56': 'WY'
}

# Folder containing the GeoPackage files
input_folder = os.path.join(data_path, "state_gpkgs") 

# Geodatabase to store the exported feature classes
output_gdb = os.path.join(data_path, "nsi_2022.gdb")  
arcpy.env.workspace = output_gdb

In [ ]:
# List to store all feature classes in all GeoPackages
feature_classes = []

# Use arcpy.da.Walk() to iterate through the subfolders and locate all GeoPackage files
for dirpath, dirnames, filenames in os.walk(input_folder):
    for filename in filenames:
        if filename.endswith(".gpkg"):
            # Full path to the GeoPackage file
            gpkg_path = os.path.join(dirpath, filename)
            
            # Walk through the contents of the GeoPackage
            walk = arcpy.da.Walk(gpkg_path)
            for dirpath_gpkg, dirnames_gpkg, filenames_gpkg in walk:
                for feature_class_name in filenames_gpkg:
                    # Append the full path to the feature_classes list
                    feature_classes.append(os.path.join(dirpath_gpkg, feature_class_name))

In [ ]:
for feature_class_path in feature_classes:
    # Extract the folder containing the GeoPackage (second deepest folder with the FIPS code)
    parent_folder = os.path.splitext(os.path.basename(os.path.dirname(feature_class_path)))[0]  # Get "nsi_2022_01" from the path
    
    # Extract the FIPS code (assuming the format is "nsi_2022_01" -> "01")
    fips_code = parent_folder.split("_")[-1]
    
    # Check if the FIPS code exists in the dictionary
    if fips_code in fips_to_abbreviation:
        # Get the state abbreviation from the FIPS code
        state_abbreviation = fips_to_abbreviation[fips_code]
        
        # Construct the new name for the feature class
        new_feature_class_name = f"{state_abbreviation}_nsi_2022"

        # convert and project to gdb
        arcpy.FeatureClassToFeatureClass_conversion(feature_class_path, output_gdb, new_feature_class_name)
        print(f"Exported {feature_class_path} to {os.path.join(output_gdb, new_feature_class_name)}")
        
    else:
        print(f"FIPS code {fips_code} not found in dictionary for {filename}")